In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

from glob import glob

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [4]:
data_txt = open('./data_split/data_dir_dict_0614.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [5]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data[i,:] = img2.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [6]:
ev_dir2 = [y for x in os.walk('/data_oh_rgb/test-images/') for y in glob(os.path.join(x[0], '*.png'))]

In [7]:
ev_data2 = np.zeros((len(ev_dir2), total_pix), dtype=np.float32)
ev_label2 = np.zeros((len(ev_dir2), 1), dtype=np.int32)

In [8]:
for i in range(len(ev_dir2)):
    img = cv2.imread(ev_dir2[i])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data2[i,:] = img2.flatten()
    if 'normal' in ev_dir2[i]:
        ev_label2[i] = 0
    else: ev_label2[i] = 1

In [11]:
f_dir1 = '/models/ICROS_re_oh/Train_0/model.ckpt-3062.meta'
f_dir2 = '/models/ICROS_re_oh/Train_0/'

#f_dir1 = '/models/ICROS_re/Train_3/model.ckpt-2280.meta'
#f_dir2 = '/models/ICROS_re/Train_3/'

ev_in = ev_data2
ev_in_label = ev_label2

saver = tf.train.import_meta_graph(f_dir1)
batch = 32
ev_steps = int(ev_in_label.shape[0]/batch)
ev_acc = np.zeros(ev_in_label.shape[0])

with tf.Session() as sess:
    
    new_saver = tf.train.import_meta_graph(f_dir1)
    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))
    graph = tf.get_default_graph()
    #for op in tf.get_default_graph().get_operations():
        #print(op.name)
        #print(op.values())
        
    data_in = graph.get_tensor_by_name("fifo_queue_DequeueUpTo:1")

    #feed_dict={data_in:ev_data[0:10], label:ev_label[0:10]}
    keep_prob = graph.get_tensor_by_name("dropout/dropout/keep_prob:0")
    for i in range(ev_steps):
        if i < (ev_steps-1):
            feed_dict={data_in:ev_in[batch*i:batch*(i+1)], keep_prob:1.0}
            label = ev_in_label[batch*i:batch*(i+1)].reshape(batch)
        else:
            feed_dict={data_in:ev_in[batch*i:], keep_prob:1.0}
            temp = ev_in_label[batch*i:]
            label = temp.reshape(len(temp))
        
        prob = graph.get_tensor_by_name("softmax_tensor:0")
        classes = graph.get_tensor_by_name("ArgMax:0")
        
        
        result_pr = sess.run(prob, feed_dict=feed_dict)
        result_cl = sess.run(classes, feed_dict=feed_dict)
        
        print(result_pr)
        print(result_cl)
        print(label)

        acc = np.equal(result_cl, label).reshape((1,len(label)))*1
        
        print(acc)
        print(np.mean(acc))
        
        if i < (ev_steps-1):
            ev_acc[batch*i:batch*(i+1)] = acc
        else:
            ev_acc[batch*i:] = acc


INFO:tensorflow:Restoring parameters from /models/ICROS_re_oh/Train_0/model.ckpt-3062
[[8.2875084e-04 9.9917126e-01]
 [2.1655478e-04 9.9978346e-01]
 [2.0792889e-03 9.9792069e-01]
 [3.8879209e-03 9.9611205e-01]
 [1.7709982e-03 9.9822897e-01]
 [1.4324773e-03 9.9856746e-01]
 [6.9637755e-03 9.9303621e-01]
 [9.0140595e-05 9.9990988e-01]
 [1.5153213e-02 9.8484683e-01]
 [8.3913496e-03 9.9160868e-01]
 [3.9499253e-03 9.9605006e-01]
 [9.5579773e-03 9.9044198e-01]
 [2.8397493e-02 9.7160250e-01]
 [3.7217524e-04 9.9962783e-01]
 [1.1604495e-03 9.9883956e-01]
 [2.0507447e-02 9.7949255e-01]
 [4.8771626e-03 9.9512285e-01]
 [2.1627059e-02 9.7837299e-01]
 [3.1997934e-03 9.9680018e-01]
 [8.9655429e-02 9.1034460e-01]
 [1.9749908e-02 9.8025012e-01]
 [2.9249391e-02 9.7075057e-01]
 [2.9410783e-03 9.9705899e-01]
 [7.7491831e-03 9.9225074e-01]
 [1.1419234e-03 9.9885809e-01]
 [8.7339890e-01 1.2660110e-01]
 [9.4470710e-01 5.5292927e-02]
 [9.8399413e-01 1.6005857e-02]
 [9.1964096e-01 8.0359057e-02]
 [8.9223218e-01

In [12]:
print(ev_acc[-1])
print(np.mean(ev_acc))

1.0
1.0
